# Data Modeling Strategies for Tanzania Water Wells

# 1. Business Understanding
Water is a basic for all living organisms including humans. Tanzania, however has struggled to provide clean water to its population of over 57 million people. Many water points have been established in the country but some are in need of repair while others do not function at all. 

This has led to some families travelling for miles to get clean water. Children will have to miss crucial school days to look for water for their families. They sometimes have to rely on unclean water and thus risk getting themselves and their families ill.

Many NGOs are willing to help. However, they have limited resources and would want to first identify the wells that need repair, those that do not function and those in operation so as to know where to start. That is where we come in.

## 1.1 Problem Statement
MajiSafi Aid has partnered with the Government of Tanzania and the local communities are on a mission to make access clean and usable water a stone-throw distance from every homestead. However Tanzania is a vast country and fixing all problems from North to South will be time-consuming and won't reach those who need it more in time.

They thus need to **identify and prioritise the non-functional wells and those that need repair**. This way, they can focus their limited resources on wells that have the most impact on the community around it.

## 1.2 Objectives
The objective is to help MajiSafiAid and the National Government to **predict the condition of various water wells** in Tanzania. We will need to classify the wells  into three categories
- **Functional** - The well is working as intended and needs no repairs.
- **Functional needs repair** - The well is working but needs some repair work done on it.
- **Non-functional** - The well does not work at all

This will enable MajiSafi Aid to:
 * To prioritize which wells they can focus to fix first depending on the well condition and their reach and impact in the society.
 * Plan their resources and budget accordingly to ensure more communities have access to clean and reliable water.
 * Improve the standards of living of many Tanzanians by providing fresh water for their everyday use.


## 1.3 Metrics of Success
The project will be termed as successful when:
- We have successully developed a machine learning model to correctly predict the condition of a well with the features provided.
- We achieve an accuracy score of 80% and a recall score of 85%

# 2. Data Understanding
The source of this dataset is [DrivenData](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table.) which was gotten from [Taarifa](http://taarifa.org/) and the [Tanzanian Ministry of Water](http://maji.go.tz/). 

The dataset contains three tables. Namely:
 - `TestSetValues.csv` - This table contains the independent variables that need predictions.
 - `TrainingSetValues.csv` - This table contains the independent variables for the training set.
 - `TrainingSetLabels.csv` - This table contains the dependent variable for each of the rows in the TrainingSetValues table.

*Only the training dataset will be used to train and test the model

`TrainingSetValues.csv` has the following 39 columns about the waterpoints:
 - `amount_tsh` - total static head. Amount of water available to the waterpoint
 - `date_recorded` - the date the waterpoint was added to the dataset
 - `funder` - name of the person/organisation funding the well
 - `gps_height` - altitude of the well
 - `installer` - name of the organization that installed the well
 - `longitude` - longitudinal position of the well
 - `latitude` - latitudinal position of the well
 - `wpt_name` - name of the well, if available
 - `num_private` - private number
 - `basin` - Geographic water basin
 - `subvillage` - well Geographic location (subvillage level)
 - `region` - well Geographic location (regional level)
 - `region_code` - regional code of the well
 - `district_code` - district code of the well
 - `lga` - local gorvenment area that the well is located
 - `ward` - ward that well is located
 - `population` - population around the well
 - `public_meeting` - True/False
 - `recorded_by` - group entering the waterpoint into the data
 - `scheme_management` - operators of the waterpoint
 - `scheme_name` - operators of the waterpoint
 - `permit` - True/False if the water point is permitted
 - `construction_year` - year that the waterpoint was constructed
 - `extraction_type` - kind of extraction that the waterpoint uses
 - `extraction_type_group` - kind of extraction that the waterpoint uses
 - `extraction_type_class` - kind of extraction that the waterpoint uses
 - `management` - organization that manages the waterpoint
 - `management_group` - general class of the organization that manages the waterpoint
 - `payment` - payment method at the waterpoint
 - `payment_type` - payment method at the waterpoint
 - `water_quality` - quality of the water at the waterpoint
 - `quality_group` - condition of the water at the waterpoint
 - `quantity` - amount of water at the waterpoint
 - `quantity_group` - amount of water at the waterpoint
 - `source` - source of the water
 - `source_type` - type of water source
 - `source_class` - class of the water source
 - `waterpoint_type` - type of the waterpoint
 - `waterpoint_type_group` - general class of the type of waterpoint

`TrainingSetLabels.csv` has the following two columns:
 - `id` - identifier of the waterpoint
 - `status_group` - this is our label/target column. It contains three values:
    - **Functional** - The well is working as intended and needs no repairs.
    - **Functional needs repair** - The well is working but needs some repair work done on it.
    - **Non-functional** - The well does not work at all

In [1]:
# Importing the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Setting pandas to show all columns and rows
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', None)

# Sklearn libraries
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, recall_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

In [2]:
values  = pd.read_csv("Data\TrainingSetValues.csv", index_col = "id")
labels = pd.read_csv("Data\TrainingSetLabels.csv", index_col = "id")

# Merging the training labels and training values into one dataset
training_data = pd.concat([values, labels], axis = 1)
training_data.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [9]:
training_data.tail()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,Pangani,Kiduruni,Kilimanjaro,3,5,Hai,Masama Magharibi,125,True,GeoData Consultants Ltd,Water Board,Losaa Kia water supply,True,1999,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,Rufiji,Igumbilo,Iringa,11,4,Njombe,Ikondo,56,True,GeoData Consultants Ltd,VWC,Ikondo electrical water sch,True,1996,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,Rufiji,Madungulu,Mbeya,12,7,Mbarali,Chimala,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,swn 80,swn 80,handpump,vwc,user-group,pay monthly,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,Rufiji,Mwinyi,Dodoma,1,4,Chamwino,Mvumi Makulu,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional
26348,0.0,2011-03-23,World Bank,191,World,38.104048,-6.747464,Kwa Mzee Lugawa,0,Wami / Ruvu,Kikatanyemba,Morogoro,5,2,Morogoro Rural,Ngerengere,150,True,GeoData Consultants Ltd,VWC,NaN,True,2002,nira/tanira,nira/tanira,handpump,vwc,user-group,pay when scheme fails,on failure,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional


**Observation**

The Train dataset is uniform from start to finish.

In [ ]:
# Shape of the dataset
print(f"The train dataset has {training_data.shape[0]} rows and {training_data.shape[1]} columns")

The train dataset has 59400 rows and 40 columns


In [ ]:
# Data Types of the train dataset
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             59400 non-null  float64
 1   date_recorded          59400 non-null  object 
 2   funder                 55765 non-null  object 
 3   gps_height             59400 non-null  int64  
 4   installer              55745 non-null  object 
 5   longitude              59400 non-null  float64
 6   latitude               59400 non-null  float64
 7   wpt_name               59400 non-null  object 
 8   num_private            59400 non-null  int64  
 9   basin                  59400 non-null  object 
 10  subvillage             59029 non-null  object 
 11  region                 59400 non-null  object 
 12  region_code            59400 non-null  int64  
 13  district_code          59400 non-null  int64  
 14  lga                    59400 non-null  object 
 15

**Observation**

The train dataset contains 10 numerical and 31 categorical columns

In [6]:
# Concise Statistical Summary
training_data.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


**Observation**
- On average, each well serves around 180 people.
- On average, each waterpoint would need to move 318 metres from it's source to their respective discharge points.

In [7]:
# Statistical Summary for Categorical Columns
training_data.describe(include="object")

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
count,59400,55765,55745,59400,59400,59029,59400,59400,59400,56066,59400,55523,31234,56344,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,356,1897,2145,37400,9,19287,21,125,2092,2,1,12,2696,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6,3
top,2011-03-15,Government Of Tanzania,DWE,none,Lake Victoria,Madukani,Iringa,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
freq,572,9084,17402,3563,10248,508,5294,2503,307,51011,59400,36793,682,38852,26780,26780,26780,40507,52490,25348,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625,32259


**Observation**
- The Government of Tanzania funds for most of the wells.
- The most common basin is Lake Victoria
- GeoData Consultants Limited recorded all of the records in this dataset.
- Most of the water in the wells are extracted by gravity
- Village Water Committee (VWC) manages most of the waterpoints in Tanzania
- Most users do not pay for the water in the wells.
- The water in most of the wells is soft, good for domestic use and enough for the community.
- Most wells have their sources as springs.



In [10]:
# Checking for unique values
for coln in training_data:
    unique_value = training_data[coln].unique()
    print(f"{coln}\n, {unique_value}\n")

amount_tsh
, [6.00e+03 0.00e+00 2.50e+01 2.00e+01 2.00e+02 5.00e+02 5.00e+01 4.00e+03
 1.50e+03 6.00e+00 2.50e+02 1.00e+01 1.00e+03 1.00e+02 3.00e+01 2.00e+03
 4.00e+02 1.20e+03 4.00e+01 3.00e+02 2.50e+04 7.50e+02 5.00e+03 6.00e+02
 7.20e+03 2.40e+03 5.00e+00 3.60e+03 4.50e+02 4.00e+04 1.20e+04 3.00e+03
 7.00e+00 2.00e+04 2.80e+03 2.20e+03 7.00e+01 5.50e+03 1.00e+04 2.50e+03
 6.50e+03 5.50e+02 3.30e+01 8.00e+03 4.70e+03 7.00e+03 1.40e+04 1.30e+03
 1.00e+05 7.00e+02 1.00e+00 6.00e+01 3.50e+02 2.00e-01 3.50e+01 3.06e+02
 8.50e+03 1.17e+05 3.50e+03 5.20e+02 1.50e+01 6.30e+03 9.00e+03 1.50e+02
 1.20e+05 1.38e+05 3.50e+05 4.50e+03 1.30e+04 4.50e+04 2.00e+00 1.50e+04
 1.10e+04 5.00e+04 7.50e+03 1.63e+04 8.00e+02 1.60e+04 3.00e+04 5.30e+01
 5.40e+03 7.00e+04 2.50e+05 2.00e+05 2.60e+04 1.80e+04 2.60e+01 5.90e+02
 9.00e+02 9.00e+00 1.40e+03 1.70e+05 2.20e+02 3.80e+04 2.50e-01 1.20e+01
 6.00e+04 5.90e+01]

date_recorded
, ['2011-03-14' '2013-03-06' '2013-02-25' '2013-01-28' '2011-07-13'
 '2011-0

# 3. Data Preparation